In [ ]:
import requests
import json
import os

In [ ]:
# ! pip install certifi

In [ ]:
import certifi

In [ ]:
srv_url = 'http://127.0.0.1:5000'
srv_url = 'https://ls-agent-cd78b4dee2fb.herokuapp.com'

### Test

In [ ]:
def test_get (msg, url = srv_url):
    url = f'{url}/{msg}'
    rsp = requests.get(url, verify=certifi.where())
    print(rsp.status_code)
    try:
        print(json.dumps(rsp.json()))
    except Exception as e:
        print(str(e))
    return

In [ ]:
def test_post (msg, path, url = srv_url):
    headers = {
    "Content-Type": "application/json"
    }
    url = f'{url}/{path}'
    response = requests.post(url, json={"question": msg}, headers=headers)
    data = response.json()
    answer = data.get('answer', 'No answer received')
    print(f"Answer: {answer}")
    return

In [ ]:
test_get('')

In [ ]:
test_post('What is Rayleigh scattering?','openai-direct')

In [ ]:
test_post('What is the shortest way to travel from San Jose to Los Angeles by car?','openai-direct')

In [ ]:
test_post('When was Microsoft founded?','openai-direct')

In [ ]:
test_post('Tell me the history of Microsoft in 100 words','openai-direct')

In [ ]:
test_post('What does EPAM software company do?','openai-direct')

In [ ]:
test_post('What is the title and authors of the paper on transformers?','openai-direct')